# Count the Number of Score Combinations

In an American football game, a play can lead to 2 points (safety), 3 points (field 
goal), or 7 points (touchdown, assuming the extra point).  Many different 
combinations of 2, 3, and 7 point plays can make up a final score.  For example,
four combinations of plays yield a score of 12:

- 6 safeties ($2\times 6 - 12$),
- 3 safeties, and 2 field goals ($2\times 3 + 3\times 2 = 12$),
- 1 safety, 1 field goal, and 1 touchdown ($2\times 1 + 3 \times 1 + 7\times 1 = 12$),
- 4 field goals ($3\times 4 = 12$).

**Write a program that takes a final score and scores for individual plays, and 
returns the number of combinations of plays that result in the final score.**

## Solution

We can gain some intuition by considering small scores.  For example, a 9 point score
can be achieved in the following ways:

- scoring 7 points, followed by a 2 point play,
- scoring 6 points, followed by a 3 point play, and
- scoring 2 points, followed by a 7 point play.

Generalizing, an $s$ spoint score can be achieved by an $s - 2$ point score, followed
by a 2 point play, an $s - 3$ point score, followed by a 3 point play, or an $s - 7$
point score, followed by a 7 point play.  This gives us a mechanism for recursively
enumerating all possible scoring sequences which lead to a given score.  Note that 
different sequences may lead to the same score combination, eg a 2 point play 
followed by a 7 point play and a 7 point play followed by a 2 point play both lead
to a final score of 9.  A brute-force approach might be to enumerate these sequences,
and count the distinct combinations within these sequences, eg by sorting each 
sequence and inserting into a hash table.

The time complexity is very high, since there may be a very large number of scoring
sequences.  Since all we care about are the combinations, a better approach is to
focus on the number of combinations for each possible number of plays of a single
type.

For example, if the final score is 12 and we are ollowed allowed 2 point plays, there 
is exactly 1 way to get 12.  Now suppose we are allowed both 2 and 3 point plays,
Since all we care about are combinations, assume the 2 point plays come before the 3
point plays.  We could have zero 2 point plays, for which there is one combination of 
3 point plays, one two point play, for which there no combinations of 3 point plays
(since 3 does not evenly divide 12 - 2, two 2 point plays, for which there no
combination of 3 point plays (since 3 evenly divides $12 - 2\times 3$), three 2
point plays, for which there is one combination of 3 point plays (since 3 evenly
divides $12 - 2\times 3$), etc.  To count combinations when we add 7 point plays
to the mix, we add the number of combinations of 2 and 3 that lead to 12 and 15 - 
these are the only scores from which 7 point plays can lead to 12.
 
Naively implemented, for the general case, ie, individual play scores are 
$W[0], W[1],...,W[i - 1]$, and $s$ the final score, the approach outlined above has
exponential complexity because it repeatedly solves the same problems.  We can use
DP to reduce its complexity.  Let the 2D array $A[i][j]$ store the number of score
combinations that result in a total of $j$, using individual plays of scores
$W[0], W[1],...,W[i - 1]$.  For example, $A[1][12]$ is the number of ways in which
we can achieve a total of 12 points, using 2 and/or 3 point plays.  Then, 
$A[i + 1][j]$ is simply $A[i][j]$ (no  $W[i + 1]$ point plays used to get to $j$), 
plus $A[i][j - W[i - 1]]$ (one $W[i + 1]$ point play), plus $A[i][j - 2W[i + 1]]$
(two $W[i + 1]$ point plays), etc.

The algorithm directly based on the above discussion consists of three nested loops.
The first loop is over the total range of scores, and the second loop is over scores 
for individual plays.  For a given $i$ and $j$, the third loop iterates over at most 
$j/W[i] + 1$ values.  (For example, if $j = 10, i = 1, \text{and } W[i] = 3$, then
the third loop examines $A[0][10], A[0][7], A[0][4], A[0][1]$.)  Therefore number
of iterations for the third loop is bounded by $s$.  Hence, the overall time 
complexity is $O(sns) = )(s^2n)$ (first loop is to $n$, second is to $s$, third is
bounded by $s$).
 
Looking more carefully at the computation for the row $A[i + 1]$, it becomes
apparent that it is not as efficient as it could be.  As an example, suppose we are
working 2 and 3 point plays.  Suppose we are done with 2 point plays.  Let $A[0]$ 
be the row holding the result for just 2 point plays, ie $A[0][j]$ is the number of 
combinations of 2 point plays that result in a final score of $j$.  The number of
score combinations to get a final score of 12 when we include 3 point plays in 
addition to 2 point plays is $A[0][0] + A[0][3] + A[0][6] + A[0][9] + A[0][12]$.
The number of score combinations to get a final score of 15 when we include 3 point
plays in addition to 2 point plays is 
$A[0][0] + A[0][3] + A[0][6] + A[0][9] + A[0][12] + A[0][15]$.  Clearly this
repeats computation - $A[0][0] + A[0][3] + A[0][6] + A[0][9] + A[0][12]$ was 
computed when considering the final score of 12.
 
Note that $A[1][15] = A[0][15] + A[1][12]$.  Therefore a better way to fill in 
$A[1]$ is as follows:  
$A[1][0] = A[0][0],A[1][1] = A[0][1],A[1][2] = A[0][2],A[1][3] = $
$A[0][3] + A[1][0],A[1][4] = A[0][4] + A[1][1],A[1][5] = A[0][5] + A[1][2], ...$.
Observe that $A[1][i]$ takes $O(1)% time to compute - it's just 
$A[0][i] + A[1][i - 3]$.  The code below implements the generalization of this 
approach:

In [3]:
def num_combinations_for_final_score(final_score: int, 
                                     individual_play_scores):
    # One way to reach 0.
    num_combinations_for_score = [[1] + [0] * final_score
                                  for _ in individual_play_scores]
    for i in range(len(individual_play_scores)):
        for j in range(1, final_score + 1):
            without_this_play = (num_combinations_for_score[i - 1][j]
                                 if i >= 1 else 0)
            with_this_play = (
                num_combinations_for_score[i][j - individual_play_scores[i]]
                if j >= individual_play_scores[i] else 0)
            num_combinations_for_score[i][j] = (
                without_this_play + with_this_play)
    return num_combinations_for_score[-1][-1]

num_combinations_for_final_score(12, [1,2,3,6])

27

The time complexity is $O(sn)$ (two loops, one to $s$, the other to $n$) and the space 
complexity is $O(sn)$ (the size of the 2D array).